# test checking of input consistency in KKRimp calculation

In [1]:
from aiida import load_profile

profile = load_profile()
import numpy as np
from aiida_kkr.workflows import combine_imps_wc
from aiida.orm import load_node, Dict, groups
# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node, StructureData 
from aiida.orm import (Group, load_group, load_node, load_code, groups,groups,
                      WorkChainNode)
from aiida.engine import submit, exceptions
from aiida.common.exceptions import NotExistent
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_imp_dos, kkr_imp_wc, kkr_startpot_wc, combine_imps_wc
import numpy as np
from aiida_kkr.calculations import KkrimpCalculation, VoronoiCalculation
import matplotlib.pyplot as plt
from aiida_kkr.tools.common_workfunctions import get_username
import matplotlib.gridspec as gridspec
from aiida_kkr.tools import plot_kkr

from aiida.engine import submit

In [2]:
dict1 = {'ss':4,'kk':20}
Dict1 = Dict(dict=dict1)

In [39]:
groups.

AttributeError: module 'aiida.orm.groups' has no attribute 'groups'

In [4]:
c1 = load_node('466c0a4d-afbb-4e48-b539-3ca53806d9ff')

In [5]:
c1_gf_wc = c1.inputs.host_Greenfunction_folder.get_incoming(node_class=combine_imps_wc).all()[0].node


/opt/aiida-core/aiida/orm/querybuilder.py:268: AiidaEntryPointWarning: Process type 'aiida_kkr.workflows._combine_imps.combine_imps_wc' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_kkr.workflows._combine_imps.combine_imps_wc' to remove this warning.
  warnings.warn(


In [6]:
c1_gf_wc.inputs.offset_imp2['index']

3

In [7]:
c1 = load_node('466c0a4d-afbb-4e48-b539-3ca53806d9ff')
builder = c1.get_builder_restart()
builder.metadata.dry_run = True
c1new = submit(builder)

11/01/2021 03:28:02 PM <2621929> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [REPORT] [115591|KkrimpCalculation|_get_and_verify_hostfiles]: impurity_info node from input and from previous GF calculation are NOT compatible!. 
11/01/2021 03:28:02 PM <2621929> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [REPORT] [115591|KkrimpCalculation|on_except]: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/plumpy/process_states.py", line 231, in execute
    result = self.run_fn(*self.args, **self.kwargs)
  File "/opt/aiida-core/aiida/engine/processes/calcjobs/calcjob.py", line 329, in run
    calc_info = self.presubmit(folder)
  File "/opt/aiida-core/aiida/engine/processes/calcjobs/calcjob.py", line 522, in presubmit
    calc_info = self.prepare_for_submission(folder)
  File "/opt/aiida-kkr/aiida_kkr/calculations/kkrimp.py", line 155, in prepare_for_submission
    tmp = self._check_and_extract_input_nodes(tempfolder)
  File "/opt/aiida-kkr

InputValidationError: impurity_info nodes (input and GF calc) are not compatible

In [8]:
c2 = load_node('90a4c68c-fb09-481a-9098-f07987b90a36')
builder = c2.get_builder_restart()
builder.metadata.dry_run = True
c2new = submit(builder)

11/01/2021 03:28:05 PM <2621929> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [REPORT] [115592|KkrimpCalculation|_get_and_verify_hostfiles]: impurity_info node from input and from previous GF calculation are NOT compatible!. 
11/01/2021 03:28:06 PM <2621929> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [REPORT] [115592|KkrimpCalculation|on_except]: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/plumpy/process_states.py", line 231, in execute
    result = self.run_fn(*self.args, **self.kwargs)
  File "/opt/aiida-core/aiida/engine/processes/calcjobs/calcjob.py", line 329, in run
    calc_info = self.presubmit(folder)
  File "/opt/aiida-core/aiida/engine/processes/calcjobs/calcjob.py", line 522, in presubmit
    calc_info = self.prepare_for_submission(folder)
  File "/opt/aiida-kkr/aiida_kkr/calculations/kkrimp.py", line 155, in prepare_for_submission
    tmp = self._check_and_extract_input_nodes(tempfolder)
  File "/opt/aiida-kkr

InputValidationError: impurity_info nodes (input and GF calc) are not compatible

In [25]:
# check for a case where the test should pass
##  3-4
c1 = load_node('90554')
builder = c1.get_builder_restart()
builder.metadata.dry_run = True

#c1_new=submit(builder)

In [26]:
# check for a case where the test should pass
##  3-3
c2 = load_node('67107')
builder = c2.get_builder_restart()
builder.metadata.dry_run = True
#c2new = submit(builder)

In [27]:
node_112700 = load_node(112700)
node_110833 = load_node(110833)

In [35]:
parent_gf_node = node_112700.inputs.gf_host_remote.get_incoming(node_class= combine_imps_wc).all()[0].node
parent_gf_node

/opt/aiida-core/aiida/orm/querybuilder.py:268: AiidaEntryPointWarning: Process type 'aiida_kkr.workflows._combine_imps.combine_imps_wc' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_kkr.workflows._combine_imps.combine_imps_wc' to remove this warning.
  warnings.warn(


<WorkChainNode: uuid: 668bf31a-d3c5-42d0-af51-6852cb98a2a3 (pk: 110833) (aiida_kkr.workflows._combine_imps.combine_imps_wc)>

In [37]:
parent_gf_node.inputs.

AttributeDict({'kkr': <Code: Remote code 'kkr' on claix18_init, pk: 206, uuid: 77705e42-72da-4e0a-b546-5bc7804b9a3b>, 'options': <Dict: uuid: 7f3dceac-b663-4362-ac9e-39480ca4e2cb (pk: 110830)>, 'params_kkr_overwrite': <Dict: uuid: f09041b3-8d05-4d91-bd40-b0d480740116 (pk: 110831)>})

In [13]:
#Delete
grps = [102, 104, 115, 118, 145, 146, 147, 148, 149, 150]
nodes_output = dict()
for grp in grps:
    nodes = list(load_group(grp).nodes)
    for nod in nodes:
        output_data = nod.outputs.JijData.get_array('JijData')
        nodes_output[nod.label] = output_data

In [14]:
lable_list = nodes_output.keys()
for lable in lable_list:
    if 'V:V' in lable:
        data = nodes_output[lable]
        print(f'label :  {lable}, -- value : {data},  D/J = {data[0][4]/data[0][3]} \n')

label :  V:V:Bi2Te3_il_3_3_Off_1, -- value : [[-0.41851489  0.          0.         11.79131331  2.22521903  0.03742837
  -1.01574039 -1.97951264]
 [ 0.41851489  0.          0.         11.79131331  2.22521903 -0.03742837
   1.01574039  1.97951264]],  D/J = 0.18871680945523397 

label :  V:V:Bi2Te3_il_3_4_Off_1, -- value : [[ 0.00000000e+00  0.00000000e+00  5.82221121e-01 -6.60883464e-01
   5.42866244e-03  4.23692634e-03 -3.39266053e-03  9.31964434e-05]
 [ 0.00000000e+00  0.00000000e+00 -5.82221121e-01 -6.60883464e-01
   5.42866244e-03 -4.23692634e-03  3.39266053e-03 -9.31964434e-05]],  D/J = -0.008214250673180446 

label :  V:V:Bi2Te3_il_3_3_Off_2, -- value : [[-0.83702979  0.          0.          0.28913975  1.7032417   0.00807675
   1.69727506 -0.14221267]],  D/J = 5.890721457021668 

label :  V:V:Bi2Te3_il_3_4_Off_2, -- value : [[-0.41851489  0.          0.58222112 -2.04632822  0.31092339  0.22145198
  -0.15413625  0.1545134 ]],  D/J = -0.15194208991279196 

label :  V:V:Bi2Te3_il_3_